# Data walking through

In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
transac = pd.read_csv("dataset/ratebeer/transac.csv", header=None)
columns = ["GiverID", "ReceiverID", "itemID", "timestamp"]
transac.columns = columns
transac

In [ ]:
number_of_row = len(transac)
# get number of products
number_of_product = len(np.unique(transac["itemID"]))

transac_grouped = transac.groupby(["GiverID", "ReceiverID", "timestamp"]).aggregate(lambda x: list(np.unique(x)))
number_of_transac = len(transac_grouped)

transac_grouped.head()

In [ ]:
_index = [i for i in range(len(transac_grouped))]
transac_grouped.index = _index
transac_grouped

In [ ]:
print("Number of row:", number_of_row)
print("Number of product:", number_of_product)
print("Number of transac:", number_of_transac)

In [ ]:
def transactionEncoder(df):
    # 'transactions' is now temporary variable
    transactions = [row["itemID"] for index, row in df.iterrows()]
    from mlxtend.preprocessing import TransactionEncoder

    transaction_encoder = TransactionEncoder()
    transac_matrix = transaction_encoder.fit_transform(transactions).astype("int")
    transac_df = pd.DataFrame(transac_matrix, columns=transaction_encoder.columns_)

    return transac_df, transac_matrix

# Transaction-based wish prediction using apriori

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

transac_df, transac_matrix = transactionEncoder(transac_grouped)

frequent_itemsets = apriori(transac_df, min_support=0.2)
print(frequent_itemsets)
# rules = association_rules(frequent_itemsets, metric="confidence")
# # Something FAILED
# rules

In [ ]:
np.unique(transac_df.sum(), return_counts=True)

In [ ]:
map_col, rmap_col = {}, {}
idx, _columns = 0, []
for val in transac_df.columns:
    map_col[val] = idx
    rmap_col[idx] = val
    _columns.append(idx)
    idx += 1

# User-based prediction using cosine similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

t = cosine_similarity(transac_df[1:2], transac_matrix[:])[0]

mx = 0
for i in t:
    if i < 1 and i > mx:
        mx = i
        print(i)
mx

In [ ]:
arr = []
for i in range(len(t)):
    arr.append((t[i], i))

arr = sorted(arr)[::-1]
arr[:20]

In [ ]:
def to_row_df(items, _columns):
    row_df = pd.DataFrame(data=[np.zeros(len(_columns)).astype(int)], columns=_columns)
    for i in items:
        row_df[i] = 1
    return row_df

def predict (utility_df, utility_matrix, return_num=5):
    import time
    start_time = time.time()
    for i in range(len(utility_matrix)):
        sim = cosine_similarity(utility_df, utility_matrix[i:i+1])[0][0]
        if 0. < sim and sim < 1.:
            return transac_grouped.iloc[i][0]
    return []
    # sim_items = cosine_similarity(utility_df, utility_matrix[:200])[0]
    # result = []
    # for i in range(len(sim_items)):
    #     if 0.5 < sim_items[i] and sim_items[i] < 1.:
    #         return transac_grouped.iloc[i][0]
    #         result.append((sim_items[i], transac_grouped.iloc[i][0]))
    # result = sorted(result)[::-1][:return_num]
    # print(time.time()-start_time)
    # return result

In [ ]:
pivot = int(0.7*len(transac_grouped))
# print(pivot)
train_set = transac_grouped[:pivot]
test_set  = transac_grouped[pivot:]
print(len(train_set))
print(len(test_set))


In [ ]:
def in_train_lst(lst, _columns):
    for i in lst:
        if i not in _columns:
            return False
    return True

def givenN_evaluate(train, test, given_num):
    import time
    start_time = time.time()
    train_df, train_matrix = transactionEncoder(train)

    score = cnt = 0
    for i in range(len(test)):
        lst = test.iloc[i][0]
        
        if len(lst) <= given_num or not in_train_lst(lst, train_df.columns):
            continue
        given_items = lst[:-given_num]
        test_items = lst[-given_num:]
        row_df = to_row_df(test_items, train_df.columns)
        suggests = predict(row_df, train_matrix)
        print("{}/{} {} - {}".format(cnt+1, i+1, lst[given_num:], suggests))
        if len(suggests):
            for s in suggests:
                if s in test_items:
                    score += 1
                    break
        cnt += 1
        # if cnt == 200:
        #     break
    print(time.time() - start_time)
    print('cnt=', cnt)
    return score

score = givenN_evaluate(train_set, test_set, 1) #given 1
score